In [2]:
%%html

<style>    
    @import url("css/custom_styles.css")
</style>

In [3]:
%matplotlib inline

#
# import utilitaires
#
import os

import helpers as hlp
import helpers.bovw as bovw
import helpers.features as feat

from sklearn.cluster import MiniBatchKMeans, DBSCAN

In [4]:
config_overrides = hlp.MetaObject.from_json("config_overrides.json")

#
# features orb
#
orb_features_config = feat.OrbFeaturesConfig()
if not orb_features_config is None:
    hlp.MetaObject.override_from_object(orb_features_config, 
                                        config_overrides.orb.features)

orb_features = feat.load(orb_features_config, None)

print("ORB features", orb_features.features.shape)

ORB features (20908751, 32)


In [5]:
#
# parametres bag of visual words
#
bovw_config = bovw.Config()
if not config_overrides is None:
    hlp.MetaObject.override_from_object(bovw_config, 
                                        config_overrides.orb.bovw)

print("bovw_config")
display(vars(bovw_config))

bovw_config


{'install_path': 'dataset/OrbBoVW.hd5',
 'read_only': True,
 'n_clusters': 200,
 'force_generate': False}

In [5]:
mbk_clusterer = MiniBatchKMeans(n_clusters=1000, batch_size=256 * os.cpu_count())

with hlp.Profile() as orb_kmeans_profile:
    mbk_clusterer.fit(orb_features.features)

print("ORB BoVW process time (MiniBatchKMeans):", orb_kmeans_profile.round_duration(), "s")
print( mbk_clusterer.cluster_centers_.shape )

ORB BoVW process time: 238.96 s
(1000, 32)


In [6]:
dbscan_clusterer = DBSCAN(n_jobs=-1)

with hlp.Profile() as orb_kmeans_profile:
    dbscan_clusterer.fit(orb_features.features)

n_clusters_ = len(set(dbscan_clusterer.labels)) - (1 if -1 in dbscan_clusterer.labels else 0)

print("ORB BoVW process time (DBSCAN):", orb_kmeans_profile.round_duration(), "s")
print( n_clusters_ )

*sklearn.cluster.KMeans* ne semble pas conçu pour gérer une très grande quantité de données. Il est mort dans d'atroces soufrances en 6 minutes entrainant avec lui dans le deuil un ordinateur et probablement plusieurs fichiers temporaires.

*sklearn.cluster.DBSCAN* est lui aussi lent si on utilise la force brute sur OrbFeatures: pas termine après 20 minutes.

Par contre *sklearn.cluster.MiniBatchKMeans* est très impressionant: tout a été ingéré en ~50s pour générer 200 clusters. Ceci dit, l'algorithme ne semble pas "scaler" linéairement avec le nombre de clusters (5000 clusters ne converge pas apres 15 minutes, 1000 clusters en 240s). Déterminer leur nombre optimale ne sera pas une mince affaire.

Il apparait clair qu'il faut resampler les features pour en extraire le BoVW.